# P-Pdot Diagram

This Google Colaboratory page generates figures related with the pulsar P-Pdot diagram. These figures are used for a review article, [Enoto T., Kisaka S., and Shibata S., "Observational diversity of magnetized neutron stars", Rep. Prog. Phys. 82 106901, 2019 (Doi: 10.1088/1361-6633/ab3def)](https://dx.doi.org/10.1088/1361-6633/ab3def).

Original tables are uploaded in [Google spreadsheet](https://docs.google.com/spreadsheets/d/1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis/edit?usp=sharing)

## Draw ATNF Pulsar Catalog

Let us draw the P-Pdot diagram based on the [ATNF pulsar catalog](http://www.atnf.csiro.au/research/pulsar/psrcat/). The version history of this catalog is listed [here](http://www.atnf.csiro.au/research/pulsar/psrcat/catalogueHistory.html).

From this catalog, please download the columns of "Name", "P0", "P1", "Assoc", "Binary", and "Type" through "Text Output" (output style = "Short csv without errors") into your local disk. Then, upload it into this Google colaboratory using the left "UPLOAD" bottun.  

In this example page, you can download the required text file from Teru's Dropbox folder in the following lines.

-- You can also find [HEASARC version](HEASarc https://heasarc.gsfc.nasa.gov/W3Browse/all/atnfpulsar.html).

-- Note. Some CSV format error could be found in the downloaded file, where you see additional ";" is included. We need to convert it to ":

!pip3.9 install google.colab

In [1]:
import os
import pandas as pd
import numpy as np
#from google.colab import files

#cmd = "rm -f atnf.csv"
#cmd = "wget --no-check-certificate -O atnf.csv 'https://www.dropbox.com/s/9t9loqhiluvjiv3/atnfpulsar_v1.60_shortwoerr.csv?dl=0'"
#os.system(cmd)

#df = pd.read_csv('atnfpulsar_v1.60_shortwoerr.csv',sep=";",header=0,skiprows=[1])
df = pd.read_csv('./atnf_all_old.csv',sep=";",header=0,skiprows=[1])
df = df.drop('Unnamed: %d' % (len(df.columns)-1),axis=1)
print(df)
for column in ["P0","P1"]:
  df[column] = df[column].replace('*',np.nan)
  df[column] = df[column].astype("float")

flag_axp = []; flag_xins = [];
for psr in df["PSR"]:
  if 'AXP' in psr:
    flag_axp.append(True)
  else:
    flag_axp.append(False)
  if 'XINS' in psr:
    flag_xins.append(True)
  else:
    flag_xins.append(False)

flag_snr = [];
for psr in df["ASSOC"]:
  if 'SNR' in psr:
    flag_snr.append(True)
  else:
    flag_snr.append(False)

flag_binary = [];
for psr in df["BINARY"]:
  if not ('*' in psr):
    flag_binary.append(True)
  else:
    flag_binary.append(False)

flag_pulsar = np.logical_and(df["P0"] > 0.0, df["P1"] > 0.0)
flag_axp = np.logical_and(flag_pulsar,flag_axp)
flag_xins = np.logical_and(flag_pulsar,flag_xins)
flag_snr = np.logical_and(flag_pulsar,flag_snr)
flag_binary = np.logical_and(flag_pulsar,flag_binary)

version_label = 'ATNF Pulsar Catalog v2.2 (%d pulsars)' % len(df)
print(df[:5])

                                                      #        NAME        P0  \
0                                                     1  J0002+6216  0.115364   
1                                                     2  J0006+1834  0.693748   
2                                                     3  J0007+7303  0.315873   
3                                                     4    J0011+08  2.552870   
4                                                     5  J0012+5431  3.025301   
...                                                 ...         ...       ...   
3631  #Acknowledge use of the ATNF Pulsar Catalogue,...         NaN       NaN   
3632  #Where practical, reference the original paper...         NaN       NaN   
3633                 #Reference listing in basic format         NaN       NaN   
3634                   #Reference listing in bbl format         NaN       NaN   
3635                                        #Thank you!         NaN       NaN   

            P1 BINARY      

TypeError: argument of type 'float' is not iterable

In [ ]:
import matplotlib.pyplot as plt

xmin = 1e-3
xmax = 1e+2
ymin = 1e-21
ymax = 1e-9

plt.style.use('https://raw.githubusercontent.com/tenoto/macsettings/master/ppdot.mplstyle')
plt.rcParams['xtick.major.pad'] = 10.0

def plot_background(flag_landscape=False,flag_text=True):
  fig, ax = plt.subplots()

  # --- Set Magnetic Field ---
  # http://www.atnf.csiro.au/research/pulsar/psrcat/psrcat_nradlp.html?type=normal&highlight=name#name
  # Bsurf = 3.2e+19 (P Pdot)^0.5
  for Bsurf in [1e+11,1e+12,1e+13,1e+14,1e+15]: # G
    x = np.logspace(np.log10(xmin),np.log10(xmax),20)
    y = (Bsurf/3.2e+19)**2 / x
    plt.plot(x,y,linestyle="--",c="#3090C7")
    x_pivot = 12.0
    if Bsurf >= 1e+12 and flag_text:
      if flag_landscape:
        ax.text(x_pivot,0.9*(Bsurf/3.2e+19)**2/x_pivot, r'$10^{%d}$ G' % (np.log10(Bsurf)), horizontalalignment='left',verticalalignment='center',
          color="#3090C7", fontsize=20, rotation=-15)
      else:
        ax.text(x_pivot,0.9*(Bsurf/3.2e+19)**2/x_pivot, r'$10^{%d}$ G' % (np.log10(Bsurf)), horizontalalignment='left',verticalalignment='center',
          color="#3090C7", fontsize=25, rotation=-30)

  Bsurf = 4.4e+13
  x = np.logspace(np.log10(xmin),np.log10(xmax),20)
  y = (Bsurf/3.2e+19)**2 / x
  plt.plot(x,y,linestyle="-",linewidth=2,c="#3090C7")
  x_pivot = 12.0
  if flag_text:
    if flag_landscape:
      ax.text(x_pivot, 1.1*(Bsurf/3.2e+19)**2/x_pivot, r'$B$cr' % (np.log10(Bsurf)), horizontalalignment='left',verticalalignment='center',color="#3090C7",fontsize=20, rotation=-10)
    else:
      ax.text(x_pivot, 1.1*(Bsurf/3.2e+19)**2/x_pivot, r'$B$cr' % (np.log10(Bsurf)), horizontalalignment='left',verticalalignment='center',color="#3090C7",fontsize=25, rotation=-30)

  # --- Set Age ---
  # http://www.atnf.csiro.au/research/pulsar/psrcat/psrcat_nradlp.html?type=normal&highlight=name#name
  # Age = P / 2Pdot
  for age in [0.1e+3, 10e+3, 1e+6, 100e+6]: #yr
    x = np.logspace(np.log10(xmin),np.log10(xmax),20)
    y = 0.5 * x / (age * 365. * 24. * 60.0**2)
    plt.plot(x,y,linestyle="--", c="#EE9A4D")

    x_pivot = 0.0020
    if (age/1e+3) >= 1000:
      text = r'$%d$ Myr' % (age/1e+6)
    elif (age/1e+3) < 1.0:
      text = r'$%.1f$ kyr' % (age/1e+3)
    else:
      text = r'$%d$ kyr' % (age/1e+3)
    if flag_text:
      if flag_landscape:
        ax.text(x_pivot, 4.3* 0.5 * x_pivot / (age * 365. * 24. * 60.0**2),
          text, color="#EE9A4D", fontsize=20, rotation=13,
          horizontalalignment='left',verticalalignment='center')
      else:
        ax.text(x_pivot, 4.3* 0.5 * x_pivot / (age * 365. * 24. * 60.0**2),
          text, color="#EE9A4D", fontsize=25, rotation=28,
          horizontalalignment='left',verticalalignment='center')

  # --- Set Lsd ---
  # Enoto et al., 2017
  # Lsd = 3.9e+35 erg/s * (Pdot/1e+11) * (P/1sec)^-3
  for Lsd in [1e+31, 1e+33, 1e+35, 1e+37]:
    x = np.logspace(np.log10(xmin),np.log10(xmax),20)
    y = (Lsd/3.9e+35) * x**3.0 * 1e-11
    plt.plot(x,y,linestyle="--", c="#8D38C9")

    x_pivot = 1e-2
    if flag_text:
      if flag_landscape:
        ax.text(x_pivot, 40.0*(Lsd/3.9e+35) * x_pivot**3.0 * 1e-11,
         r'$10^{%d}$ erg s$^{-1}$' % (np.log10(Lsd)),
         horizontalalignment='left',  verticalalignment='center',
         color="#8D38C9", fontsize=20, rotation=43)
      else:
        ax.text(x_pivot, 40.0*(Lsd/3.9e+35) * x_pivot**3.0 * 1e-11,
         r'$10^{%d}$ erg s$^{-1}$' % (np.log10(Lsd)),
         horizontalalignment='left',  verticalalignment='center',
         color="#8D38C9", fontsize=25, rotation=60)

  # --- Death Line
  # 4 log B - 7.5 log P = 49.3
  # Chen 1993; # Zhang 2000
  #for period in [0.001, 100.0]:
  #	pdot = 10**( 2 * math.log10( period ) - 16.52 )
  #	graphDeathLine.SetPoint(i, period, pdot)

  ### Chen and Ruderman 1993
  ### http://adsabs.harvard.edu/abs/1993ApJ...402..264C
  ### 4 log B - 6.5 log P = 45.7 (Eq. 8)
  ### 4 log B - 7.5 log P = 49.3 (Eq. 6)

  x = np.logspace(np.log10(xmin),np.log10(xmax),20)
  y = 10**(2*np.log10(x)-16.52)
  plt.plot(x,y,linestyle="-", linewidth=2, c="#FFA500")

  #ax.set_title(version_label,fontsize=20)
  ax.grid(True,linestyle='--')
  ax.set_yscale('log')
  ax.set_xscale('log')
  ax.set_xlim(xmin,xmax)
  ax.set_ylim(ymin,ymax)
  ax.set_xlabel('Period (s)',fontsize=26,labelpad=12)
  ax.set_ylabel('Period Derivative (s s$^{-1}$)',fontsize=26,labelpad=12)
  ax.legend(loc='lower right',scatterpoints=1,fontsize=20)

  return fig, ax

In [ ]:
fig, ax = plot_background()

ax.scatter(df["P0"][flag_snr],df["P1"][flag_snr],
           label='SNR (%d)' % sum(flag_snr),
           marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
ax.scatter(df["P0"][flag_binary],df["P1"][flag_binary],
           label='Binary (%d)' % sum(flag_binary),
           marker='s',edgecolors='#488AC7',c="white",linewidth=2,s=150)
ax.scatter(df["P0"][flag_pulsar],df["P1"][flag_pulsar],
           label='Pulsar (%d)' % sum(flag_pulsar),
           marker='.',edgecolors='k',c="#808080",linewidth=0.5,s=100)
ax.scatter(df["P0"][flag_axp],df["P1"][flag_axp],
           label='Magnetar ATNF (%d)' % sum(flag_axp),
           marker="o", edgecolors='k', c="#FF0000", linewidth=2,s=200)
ax.scatter(df["P0"][flag_xins],df["P1"][flag_xins],
           label='XINS (%d)' % sum(flag_xins),
           marker="p", edgecolors='k', c="#E8A317", linewidth=2,s=260)
ax.set_title(version_label,fontsize=20)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot.pdf')

## Switch magnetars to the McGill Catalog

The magnetar class in the ATNF catalog is sometimes obsolete. Then, as a next step, let us switch the magnetar class to [the McGill Catalog](http://www.physics.mcgill.ca/~pulsar/magnetar/main.html). The CSV file can be downloaded from [here](http://www.physics.mcgill.ca/~pulsar/magnetar/TabO1.csv).

In order to further modify the magnetar database to the latest results, here we prepared the [Google spreadsheet](https://docs.google.com/spreadsheets/d/1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis/edit#gid=0).

In [ ]:
import os
def wget_gsheet(sheet_key,sheet_gid,outfile):
  cmd = 'wget "https://docs.google.com/spreadsheets/d/%s/export?format=csv&gid=%s" -O %s' % (sheet_key,sheet_gid,outfile)
  print(cmd);os.system(cmd)

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","0","magnetar.csv")
df_mcgill = pd.read_csv('magnetar.csv',header=1)

for column in ["P (sec)","Pdot (s/s)"]:
  df_mcgill[column] = df_mcgill[column].replace('',np.nan)
  df_mcgill[column] = df_mcgill[column].astype("float")

flag_snr_mcgill = [];
for assoc in df_mcgill["Assoc"]:
  if type(assoc) == float:
    flag_snr_mcgill.append(False)
    continue
  if 'SNR' in assoc:
    flag_snr_mcgill.append(True)
  else:
    flag_snr_mcgill.append(False)

flag_axp_mcgill =  np.logical_and(df_mcgill["P (sec)"] > 0.0, df_mcgill["Pdot (s/s)"] > 0.0)
df_mcgill[flag_snr_mcgill]
flag_axp_mcgill_snr =  np.logical_and(flag_axp_mcgill, flag_snr_mcgill)

flag_pulsar_woaxp = np.logical_and(flag_pulsar, ~flag_axp)
flag_snr_woaxp = np.logical_and(flag_snr,~flag_axp)

version_label = 'ATNF Pulsar Catalog v1.60 (%d pulsars) + McGill Magnetar' % len(df)

In [ ]:
fig, ax = plot_background()

ax.scatter(df["P0"][flag_snr_woaxp],df["P1"][flag_snr_woaxp],
           label='SNR (%d)' % (sum(flag_snr_woaxp)+sum(flag_axp_mcgill_snr)),
           marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill_snr],df_mcgill["Pdot (s/s)"][flag_axp_mcgill_snr],
           label='',
           # label='SNR Magnetar (%d)' % sum(flag_axp_mcgill_snr),
           marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
ax.scatter(df["P0"][flag_binary],df["P1"][flag_binary],
           label='Binary (%d)' % sum(flag_binary),
           marker='s',edgecolors='#488AC7',c="white",linewidth=2,s=150)
ax.scatter(df["P0"][flag_pulsar_woaxp],df["P1"][flag_pulsar_woaxp],
           label='Pulsar (%d)' % sum(flag_pulsar_woaxp),
           marker='.',edgecolors='k',c="#808080",linewidth=0.5,s=100)
ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill],df_mcgill["Pdot (s/s)"][flag_axp_mcgill],
           label='Magnetar (%d)' % sum(flag_axp_mcgill),
           marker="o", edgecolors='k', c="#FF0000", linewidth=2,s=200)
ax.scatter(df["P0"][flag_xins],df["P1"][flag_xins],
           label='XINS (%d)' % sum(flag_xins),
           marker="p", edgecolors='k', c="#E8A317", linewidth=2,s=260)
ax.set_title(version_label,fontsize=20)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot.pdf')

## Adding CCO and modifying XINS List
The above P-Pdot diagram does not include the CCO class, and the XINS class is not accurate. Let us modify these two classes using Teru's local Google spreadsheet.

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1124730326","cco.csv")
df_cco = pd.read_csv("cco.csv",header=1)

for column in ["P (sec)","Pdot (s/s)"]:
  df_cco[column] = df_cco[column].replace('',np.nan)
  df_cco[column] = df_cco[column].astype("float")

flag_cco =  np.logical_and(df_cco["P (sec)"] > 0.0, df_cco["Pdot (s/s)"] > 0.0)


wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","306748955","xins.csv")
df_xins = pd.read_csv("xins.csv",header=1)

for column in ["P (sec)","Pdot (s/s)"]:
  df_xins[column] = df_xins[column].replace('',np.nan)
  df_xins[column] = df_xins[column].astype("float")

flag_xins =  np.logical_and(df_xins["P (sec)"] > 0.0, df_xins["Pdot (s/s)"] > 0.0)

In [ ]:
fig, ax = plot_background()


ax.scatter(df["P0"][flag_snr_woaxp],df["P1"][flag_snr_woaxp],
           label='SNR (%d)' % (sum(flag_snr_woaxp)+sum(flag_axp_mcgill_snr)),
           marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill_snr],df_mcgill["Pdot (s/s)"][flag_axp_mcgill_snr],
           label='',
           # label='SNR Magnetar (%d)' % sum(flag_axp_mcgill_snr),
           marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
ax.scatter(df["P0"][flag_binary],df["P1"][flag_binary],
           label='Binary (%d)' % sum(flag_binary),
           marker='s',edgecolors='#488AC7',c="white",linewidth=2,s=150)
ax.scatter(df["P0"][flag_pulsar_woaxp],df["P1"][flag_pulsar_woaxp],
           label='Pulsar (%d)' % sum(flag_pulsar_woaxp),
           marker='.',edgecolors='k',c="#808080",linewidth=0.5,s=100)
ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill],df_mcgill["Pdot (s/s)"][flag_axp_mcgill],
           label='Magnetar (%d)' % sum(flag_axp_mcgill),
           marker="o", edgecolors='k', c="#FF0000", linewidth=2,s=200)
ax.scatter(df_xins["P (sec)"][flag_xins],df_xins["Pdot (s/s)"][flag_xins],
           label='XINS (%d)' % sum(flag_xins),
           marker="p", edgecolors='k', c="#E8A317", linewidth=2,s=260)
ax.scatter(df_cco["P (sec)"][flag_cco],df_cco["Pdot (s/s)"][flag_cco],
           label='CCO (%d)' % sum(flag_cco),
           marker="s", edgecolors='k', c="#0000FF", linewidth=2,s=230)
ax.set_title(version_label,fontsize=20)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot.pdf')

## High-B pulsars emitting X-rays

Here we add High-B pulsars which emit X-ray radiation.

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1610841382","hbp.csv")
df_hbp = pd.read_csv("hbp.csv",header=1)

for column in ["P (sec)","Pdot (s/s)"]:
  df_hbp[column] = df_hbp[column].replace('',np.nan)
  df_hbp[column] = df_hbp[column].astype("float")

flag_hbp =  np.logical_and(df_hbp["P (sec)"] > 0.0, df_hbp["Pdot (s/s)"] > 0.0)

In [ ]:
def plot_pulsars(fig,ax,flag_color=True):
  if flag_color:
    ax.scatter(df["P0"][flag_snr_woaxp],df["P1"][flag_snr_woaxp],
               label='SNR',
               marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
  else:
    ax.scatter(df["P0"][flag_snr_woaxp],df["P1"][flag_snr_woaxp],
               label='',
               marker='o',edgecolors='#808080',c="white",linewidth=2,s=320)

  if flag_color:
    ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill_snr],df_mcgill["Pdot (s/s)"][flag_axp_mcgill_snr],
               label='',
               marker='o',edgecolors='#4AA02C',c="white",linewidth=2,s=320)
  else:
    ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill_snr],df_mcgill["Pdot (s/s)"][flag_axp_mcgill_snr],
               label='',
               marker='o',edgecolors='#808080',c="white",linewidth=2,s=320)

  if flag_color:
    ax.scatter(df["P0"][flag_binary],df["P1"][flag_binary],
               label='Binary',
               marker='s',edgecolors='#488AC7',c="white",linewidth=2,s=150)
  else:
    ax.scatter(df["P0"][flag_binary],df["P1"][flag_binary],
               label='',
               marker='s',edgecolors='#808080',c="white",linewidth=2,s=150)

  if flag_color:
    ax.scatter(df["P0"][flag_pulsar_woaxp],df["P1"][flag_pulsar_woaxp],
               label='Pulsar',
               marker='.',edgecolors='k',c="#808080",linewidth=0.5,s=100)
  else:
    ax.scatter(df["P0"][flag_pulsar_woaxp],df["P1"][flag_pulsar_woaxp],
               label='',
               marker='.',edgecolors='k',c="#808080",linewidth=0.5,s=100)

  if flag_color:
    ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill],df_mcgill["Pdot (s/s)"][flag_axp_mcgill],
               label='Magnetar',
               marker="o", edgecolors='k', c="#FF0000", linewidth=2,s=200)
  else:
    ax.scatter(df_mcgill["P (sec)"][flag_axp_mcgill],df_mcgill["Pdot (s/s)"][flag_axp_mcgill],
             label='',
             marker="o", edgecolors='k', c="white", linewidth=2,s=200)

  if flag_color:
    ax.scatter(df_xins["P (sec)"][flag_xins],df_xins["Pdot (s/s)"][flag_xins],
               label='XINS',
               marker="p", edgecolors='k', c="#E8A317", linewidth=2,s=260)
  else:
    ax.scatter(df_xins["P (sec)"][flag_xins],df_xins["Pdot (s/s)"][flag_xins],
             label='',
             marker="p", edgecolors='k', c="white", linewidth=2,s=260)

  if flag_color:
    ax.scatter(df_cco["P (sec)"][flag_cco],df_cco["Pdot (s/s)"][flag_cco],
               label='CCO',
               marker="s", edgecolors='k', c="#0000FF", linewidth=2,s=230)
  else:
    ax.scatter(df_cco["P (sec)"][flag_cco],df_cco["Pdot (s/s)"][flag_cco],
               label='',
               marker="s", edgecolors='k', c="white", linewidth=2,s=230)

  if flag_color:
    ax.scatter(df_hbp["P (sec)"][flag_hbp],df_hbp["Pdot (s/s)"][flag_hbp],
               label='HBP (X-ray)',
               marker="D", edgecolors='k', c="#8D38C9", linewidth=2,s=180)
  else:
    ax.scatter(df_hbp["P (sec)"][flag_hbp],df_hbp["Pdot (s/s)"][flag_hbp],
               label='',
               marker="D", edgecolors='k', c="white", linewidth=2,s=180)
  #ax.set_title(version_label,fontsize=20)
  if flag_color:
    ax.legend(loc='lower right',scatterpoints=1,fontsize=20)

In [ ]:
fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax)
plt.savefig('ppdot_all.pdf',bbox_inches='tight',transparent=True)

In [ ]:
files.download('ppdot_all.pdf')

## Landscale plot


In [ ]:
plt.style.use('https://raw.githubusercontent.com/tenoto/macsettings/master/ppdot_landscape.mplstyle')

fig, ax = plot_background(flag_landscape=True)
plot_pulsars(fig=fig,ax=ax)
plt.savefig('ppdot_landscale.pdf',bbox_inches='tight')

plt.style.use('https://raw.githubusercontent.com/tenoto/macsettings/master/ppdot.mplstyle')

In [ ]:
files.download('ppdot_landscale.pdf')

## Gray scale figure and then adding glitch information

In [ ]:
fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax,flag_color=False)
plt.savefig('ppdot_gray.pdf',bbox_inches='tight')

In [ ]:
files.download('ppdot_gray.pdf')

Download the glitch data. P and Pdot values of the glitch data are modified for following magnetars (1E_1841-045, 1E_1048.1-5937, 1RXS_J1708-4009,
4U_0142+61, and 1E_2259+586).

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","585426704","glitch.csv")
dataset_glitch = pd.read_csv("glitch.csv", header=1)

glitch_P = []; glitch_Pdot = []; glitch_deltaF = []
size_norm = 70.0
size_offset = 6.0
for key, row in dataset_glitch.iterrows():
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    glitch_P.append(float(row['P (sec)']))
    glitch_Pdot.append(float(row['Pdot (s/s)']))
    glitch_deltaF.append(size_norm*(size_offset+np.log10(float(row['Delta_F (1e-6)']))))

fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax,flag_color=False)
ax.scatter(np.array(glitch_P),np.array(glitch_Pdot),s=np.array(glitch_deltaF),
           label='Glitch',
           c="#FF0000",edgecolor='k')
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)

plt.savefig('ppdot_glitch.pdf',bbox_inches='tight')

In [ ]:
files.download('ppdot_glitch.pdf')

## RRAT, Mode change, Nuller, and Intermittent


In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1732409989","rrat.csv")
dataset_rrat = pd.read_csv("rrat.csv", header=1)
rrat_p    = []; rrat_pdot    = []
for key, row in dataset_rrat.iterrows():
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    rrat_p.append(float(row['P (sec)']))
    rrat_pdot.append(float(row['Pdot (s/s)']))

fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax,flag_color=False)
ax.scatter(np.array(rrat_p),np.array(rrat_pdot),
           label="RRAT",
           marker="d", edgecolors='k', c="red",linewidth=2, s=300)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_rrat.pdf',bbox_inches='tight')
files.download('ppdot_rrat.pdf')

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1461529538","modechange.csv")
dataset_modechange = pd.read_csv("modechange.csv", header=1)
modechange_p    = []; modechange_pdot    = []
for key, row in dataset_modechange.iterrows():
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    modechange_p.append(float(row['P (sec)']))
    modechange_pdot.append(float(row['Pdot (s/s)']))

fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax,flag_color=False)
ax.scatter(np.array(modechange_p),np.array(modechange_pdot),
           label="Mode change + Nuller",
           marker="d", edgecolors='k', c="red",linewidth=2, s=300)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_modechange_nuller.pdf',bbox_inches='tight')
files.download('ppdot_modechange_nuller.pdf')


In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1500065469","intermittent.csv")
dataset_intermittent = pd.read_csv("intermittent.csv", header=1)
intermittent_p    = []; intermittent_pdot    = []
for key, row in dataset_intermittent.iterrows():
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    intermittent_p.append(float(row['P (sec)']))
    intermittent_pdot.append(float(row['Pdot (s/s)']))

fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax,flag_color=False)
ax.scatter(np.array(intermittent_p),np.array(intermittent_pdot),
           label="Intermittent",
           marker="d", edgecolors='k', c="red",linewidth=2, s=300)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_intermittent.pdf',bbox_inches='tight')
files.download('ppdot_intermittent.pdf')

## Giant Radio Pulses

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","366806027","grp.csv")
dataset_grp = pd.read_csv("grp.csv", header=1)

grp_p = []; grp_pdot = [];
for key, row in dataset_grp.iterrows():
	period = float(row['P (sec)'])
	pdot   = float(row['Pdot (s/s)'])
	if period > 0 and pdot > 0:
		grp_p.append(period)
		grp_pdot.append(pdot)

fig, ax = plot_background()
plot_pulsars(fig=fig,ax=ax,flag_color=False)
ax.scatter(np.array(grp_p),np.array(grp_pdot),
           label="Giant Radio Pulse",
           marker="d", edgecolors='k', c="red",linewidth=2, s=300)
ax.legend(loc='lower right',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_grp.pdf',bbox_inches='tight')
files.download('ppdot_grp.pdf')

## Zoom-up view

In [ ]:
xmin_zoom = 0.1
xmax_zoom = 30
ymin_zoom = 1e-15
ymax_zoom = 1e-9

fig, ax = plot_background(flag_text=False)
ax.set_xlim(xmin_zoom,xmax_zoom)
ax.set_ylim(ymin_zoom,ymax_zoom)
plot_pulsars(fig=fig,ax=ax)
ax.scatter(np.array(rrat_p),np.array(rrat_pdot),c="#ADD8E6",edgecolor='k',
           marker="^", linewidth='2',s=200, label='RRAT')
ax.legend(loc='upper left',scatterpoints=1,fontsize=15)
plt.savefig('ppdot_zoom.pdf',bbox_inches='tight')
files.download('ppdot_zoom.pdf')

## Transient behaviours


In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1733027220","radiomagnetar.csv")
dataset_radiomagnetar = pd.read_csv("radiomagnetar.csv", header=1)
radiomagnetar_p  = []; radiomagnetar_pdot  = []
for key, row in dataset_radiomagnetar.iterrows():
  if '<' in str(row['Pdot (s/s)']):
    continue
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    radiomagnetar_p.append(float(row['P (sec)']))
    radiomagnetar_pdot.append(float(row['Pdot (s/s)']))

wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","185595326","transient.csv")
dataset_transient = pd.read_csv("transient.csv", header=1)
transient_p  = []; transient_pdot  = []
for key, row in dataset_transient.iterrows():
  if '<' in str(row['Pdot (s/s)']):
    continue
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    transient_p.append(float(row['P (sec)']))
    transient_pdot.append(float(row['Pdot (s/s)']))

wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1394748107","giantflare.csv")
dataset_giantflare = pd.read_csv("giantflare.csv", header=1)
giantflare_p  = []; giantflare_pdot  = []
for key, row in dataset_giantflare.iterrows():
  if '<' in str(row['Pdot (s/s)']):
    continue
  if float(row['P (sec)'])>0.0 and float(row['Pdot (s/s)'])>0.0:
    giantflare_p.append(float(row['P (sec)']))
    giantflare_pdot.append(float(row['Pdot (s/s)']))

fig, ax = plot_background(flag_text=False)
ax.set_xlim(xmin_zoom,xmax_zoom)
ax.set_ylim(ymin_zoom,ymax_zoom)
ax.scatter(np.array(radiomagnetar_p),np.array(radiomagnetar_pdot),c="#D6DBDF",
  marker="*", linewidth='2',s=2500,label="Radio-emitting Magnetar")

plot_pulsars(fig=fig,ax=ax,flag_color=False)

ax.scatter(np.array(rrat_p),np.array(rrat_pdot),c="#FFFFFF",edgecolor='k',
           marker="^", linewidth='2',s=200)
ax.scatter(np.array(giantflare_p),np.array(giantflare_pdot),c="#FF0000",edgecolor='k',
  marker="*", linewidth='2',s=1020,label="Giant Flare")
ax.scatter(np.array(transient_p),np.array(transient_pdot),c="#FF0000",edgecolor='k',
  marker="o", linewidth='2',s=250, label="X-ray Outburst")
ax.legend(loc='upper left',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_zoom_transient.pdf',bbox_inches='tight')
files.download('ppdot_zoom_transient.pdf')


## Pulsar evolution (empirical)

In [ ]:
def drange(start, stop, step):
	r = start
	while r < stop:
		yield r
		r += step

# ======================
# Pulsar Evolution
# ======================
#MomentOfInertia = 1.61e+45 # g/cm2
MomentOfInertia = 1e+45 # g/cm2
VelocityOfLight = 3e+10 # cm/s
#RadiusOfNS      = 1.2e+6  # cm
RadiusOfNS      = 1e+6  # cm
factor          = 8 * np.pi**2 * RadiusOfNS**6 / (3 * MomentOfInertia * VelocityOfLight**3)

kyr = 1e+3 * 365 * 24 * 60 * 60 # sec
Myr = 1e+3 * kyr				# sec
Gyr = 1e+3 * Myr				# sec

def getPeriodPdotBfield(t, Pi, Bi, tau):
	period = np.sqrt(Pi**2 + factor * Bi**2 * tau * (1-np.exp(-2*t/tau)))
	B = Bi * np.exp(-t/tau)
	pdot   = factor * B**2  / period
	return period, pdot, B

graphEvolution = []
timeArray  = [1.0]
timeArray += [ (10**i)*kyr for i in drange(0, 10, 0.001)]
Bi_list = [1e+12, 1e+13, 1e+14, 1e+15]
for Bi in Bi_list:
	path_x = []
	path_y = []
	for time in timeArray:
		period, pdot, B = getPeriodPdotBfield(time, Pi=0.001, Bi=Bi, tau=100.0*Myr/(Bi/1.0e+12)**1.5)
		path_x.append(period)
		path_y.append(pdot)
	graphEvolution.append([path_x,path_y])

def plot_evolution(ax):
  for i in range(len(Bi_list)):
    ax.plot(np.array(graphEvolution[i][0]),np.array(graphEvolution[i][1]),
    linestyle="-",color="gray",linewidth=4)

In [ ]:
fig, ax = plot_background(flag_text=False)
plot_evolution(ax=ax)
ax.set_xlim(xmin_zoom,xmax_zoom)
ax.set_ylim(ymin_zoom,ymax_zoom)
plot_pulsars(fig=fig,ax=ax)
ax.legend(loc='upper left',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_zoom_evolution.pdf',bbox_inches='tight')
#files.download('ppdot_zoom_evolution.pdf')

## Braking Index

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1790375017","braking_index.csv")
dataset_bk = pd.read_csv("braking_index.csv", header=1)

xmin_zoom2 = 1e-2
xmax_zoom2 = 1e+2
ymin_zoom2 = 1e-15
ymax_zoom2 = 1e-9

fig, ax = plot_background(flag_text=False)
plot_evolution(ax=ax)
ax.set_xlim(xmin_zoom2,xmax_zoom2)
ax.set_ylim(ymin_zoom2,ymax_zoom2)
plot_pulsars(fig=fig,ax=ax,flag_color=False)

for key, row in dataset_bk.iterrows():
	if row['P (sec)'] > 0.0:
		#print(key,row[0],row['n'],row['P'],row['Pdot'])
		slope = 2. - float(row['n'])

		u = 1.0
		v = slope
		norm = np.sqrt(u**2+v**2)
		u = u/norm
		v = v/norm
		#print(u,v,row['n'],slope)
		plt.quiver(row['P (sec)'],row['Pdot (s/s)'],
			u, v, angles='uv',
			units='inches',scale_units='inches',scale=1.2,edgecolor='k',
			linewidth=1,color="#FF0000") #,
			#units='width',scale_units='width',scale=0.01)

		if row['n2'] != '':
			#print(key,row[0],row['n2'],row['P'],row['Pdot'])
			slope = 2. - float(row['n2'])

			u = 1.0
			v = slope
			norm = np.sqrt(u**2+v**2)
			u = u/norm
			v = v/norm
			#print(u,v,row['n'],slope)
			plt.quiver(row['P (sec)'],row['Pdot (s/s)'],
				u, v, angles='uv',
				units='inches',scale_units='inches',scale=1.2, edgecolor='k',
				linewidth=1,color="#E8A317") #,
				#units='width',scale_units='width',scale=0.01)

#ax.legend(loc='upper left',scatterpoints=1,fontsize=20)
plt.savefig('ppdot_zoom_evolution_index.pdf',bbox_inches='tight')
files.download('ppdot_zoom_evolution_index.pdf')

## P-Pdot diagram for multiwavelength observations

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1096978210","multiband.csv")
dataset_multiband = pd.read_csv("multiband.csv", header=1)
radio_p = []; radio_pdot = [];
optical_p = []; optical_pdot = [];
xray_p = []; xray_pdot = [];
gamma_p = []; gamma_pdot = [];
for key, row in dataset_multiband.iterrows():
  if row['Energy Band']=="Radio":
    radio_p.append(float(row['P (sec)']))
    radio_pdot.append(float(row['Pdot (s/s)']))
  if row['Energy Band']=="Optical":
    optical_p.append(float(row['P (sec)']))
    optical_pdot.append(float(row['Pdot (s/s)']))
  if row['Energy Band']=="X-ray":
    xray_p.append(float(row['P (sec)']))
    xray_pdot.append(float(row['Pdot (s/s)']))
  if row['Energy Band']=="γ-ray":
    gamma_p.append(float(row['P (sec)']))
    gamma_pdot.append(float(row['Pdot (s/s)']))

fig = plt.figure(figsize=(11.0,15.0))
plt.subplots_adjust(left=0.10, right=0.96, top=0.96, bottom=0.10)
ax  = fig.add_subplot(1,1,1)
ax.scatter(np.array(np.log10(radio_p)),np.array(np.log10(radio_pdot)),c="#808080",marker=".",linewidth=0.5,s=150,label="Radio")
ax.scatter(np.array(np.log10(optical_p)),np.array(np.log10(optical_pdot)),c="#0000FF",marker="s",linewidth=2,s=300,edgecolor="k",label="Optical")
ax.scatter(np.array(np.log10(xray_p)),np.array(np.log10(xray_pdot)),c="#8D38C9",marker="o",linewidth=2,s=150,edgecolor="k",label="X-ray")
ax.scatter(np.array(np.log10(gamma_p)),np.array(np.log10(gamma_pdot)),c="#FF0000",marker="D",linewidth=2,edgecolor="k",s=100,label="Gamma-ray")

ax.set_xlabel('Log Period (s)',fontsize=24, labelpad=8)
ax.set_ylabel('Log Period Derivative (s s$^{-1}$)',fontsize=24, labelpad=8)
ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=14)
ax.grid(True)
ax.set_xlim(-3,2)
ax.set_ylim(-21,-9)
ax.legend(loc='upper left',scatterpoints=1,fontsize=20)

plt.savefig('PPdot_type.pdf',bbox_inches='tight')
files.download('PPdot_type.pdf')

In [ ]:
wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","704444321","LxLsd.csv")
dataset_LxLsd = pd.read_csv("LxLsd.csv", header=1)
radio_Lsd = []; radio_Lnth = [];
optical_Lsd = []; optical_Lnth = [];
xray_Lsd = []; xray_Lnth = [];
gamma_Lsd = []; gamma_Lnth = [];
for key, row in dataset_LxLsd.iterrows():
  if row['Energy Band']=="Radio":
    radio_Lsd.append(float(row['Lsd (erg/s)']))
    radio_Lnth.append(float(row['Lnth (erg/s)']))
  if row['Energy Band']=="Optical":
    optical_Lsd.append(float(row['Lsd (erg/s)']))
    optical_Lnth.append(float(row['Lnth (erg/s)']))
  if row['Energy Band']=="X-ray":
    xray_Lsd.append(float(row['Lsd (erg/s)']))
    xray_Lnth.append(float(row['Lnth (erg/s)']))
  if row['Energy Band']=="γ-ray":
    gamma_Lsd.append(float(row['Lsd (erg/s)']))
    gamma_Lnth.append(float(row['Lnth (erg/s)']))

fig = plt.figure(figsize=(11.0,15.0))
plt.subplots_adjust(left=0.10, right=0.96, top=0.96, bottom=0.10)
ax  = fig.add_subplot(1,1,1)

ax.plot(np.array([xmin,xmax]),np.array([xmin,1.0*xmax]),c="#808080",linestyle="--")
ax.plot(np.array([xmin,xmax]),np.array([xmin-3,xmax-3]),c="#808080",linestyle="--")
ax.plot(np.array([xmin,xmax]),np.array([xmin-6,xmax-6]),c="#808080",linestyle="--")
ax.plot(np.array([xmin,xmax]),np.array([xmin-9,xmax-9]),c="#808080",linestyle="--")

ax.text(37.0,37.9,r'efficiency $\epsilon=1$',horizontalalignment='left',verticalalignment='center',fontsize=25, rotation=45)
ax.text(38.0,35.3,r'$\epsilon=10^{-3}$',horizontalalignment='left',verticalalignment='center',fontsize=25, rotation=45)
ax.text(38.0,32.3,r'$\epsilon=10^{-6}$',horizontalalignment='left',verticalalignment='center',fontsize=25, rotation=45)
ax.text(38.0,29.3,r'$\epsilon=10^{-9}$',horizontalalignment='left',verticalalignment='center',fontsize=25, rotation=45)

ax.scatter(np.array(np.log10(radio_Lsd)),np.array(np.log10(radio_Lnth)),c="#808080",marker=".",linewidth=0.5,s=150,label="Radio")
ax.scatter(np.array(np.log10(optical_Lsd)),np.array(np.log10(optical_Lnth)),c="#0000FF",marker="s",linewidth=2,s=300,edgecolor="k",label="Optical")
ax.scatter(np.array(np.log10(xray_Lsd)),np.array(np.log10(xray_Lnth)),c="#8D38C9",marker="o",linewidth=2,s=150,edgecolor="k",label="X-ray")
ax.scatter(np.array(np.log10(gamma_Lsd)),np.array(np.log10(gamma_Lnth)),c="#FF0000",marker="D",linewidth=2,edgecolor="k",s=100,label="Gamma-ray")

ax.set_xlabel('Log Spin-down Luminosity (erg s$^{-1}$)',fontsize=24, labelpad=8) #,fontweight='bold')
ax.set_ylabel('Log Luminosity (erg s$^{-1}$)',fontsize=24, labelpad=8)
ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=14)
ax.grid(True)
ax.set_xlim(28,40)
ax.set_ylim(24,40)
ax.legend(loc='upper left',scatterpoints=1,fontsize=20)

plt.savefig('LsdLband.pdf',bbox_inches='tight')
files.download('LsdLband.pdf')

## Neutron star cooling curve

In [ ]:
import scipy.interpolate

wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","1531486312","cooling.csv")
dataset_cooling = pd.read_csv("cooling.csv", header=1)

nth_age = []; nth_L = []
mag_age = []; mag_L = []
xins_age = []; xins_L = []
bbp_age = []; hbp_L = []
cco_age = []; cco_L = []
rpp_age = []; rpp_L = []


for key, row in dataset_cooling.iterrows():
  if row['Type']=="RPP" and row['L (thermal or non-thermal)']=="Non-thermal":
    nth_age.append(float(row['log10 Age (yr)'])-3.0)
    nth_L.append(float(row['log10 L (erg/s)']))
  if row['Type']=="MAG" and row['L (thermal or non-thermal)']=="Thermal":
    mag_age.append(float(row['log10 Age (yr)'])-3.0)
    mag_L.append(float(row['log10 L (erg/s)']))
  if row['Type']=="XINS" and row['L (thermal or non-thermal)']=="Thermal":
    xins_age.append(float(row['log10 Age (yr)'])-3.0)
    xins_L.append(float(row['log10 L (erg/s)']))
  if row['Type']=="HB" and row['L (thermal or non-thermal)']=="Thermal":
    bbp_age.append(float(row['log10 Age (yr)'])-3.0)
    hbp_L.append(float(row['log10 L (erg/s)']))
  if row['Type']=="CCO" and row['L (thermal or non-thermal)']=="Thermal":
    cco_age.append(float(row['log10 Age (yr)'])-3.0)
    cco_L.append(float(row['log10 L (erg/s)']))
  if row['Type']=="RPP" and row['L (thermal or non-thermal)']=="Thermal":
    rpp_age.append(float(row['log10 Age (yr)'])-3.0)
    rpp_L.append(float(row['log10 L (erg/s)']))

wget_gsheet("1sS95OZYOFYg9MxHtft2KVXxQHdPxBPyoFTdFGgSLhis","48536337","cooling_theory.csv")
dataset_cooling_theory = pd.read_csv("cooling_theory.csv", header=1)

sp    = scipy.interpolate.InterpolatedUnivariateSpline(dataset_cooling_theory['log10 Age (yr)']-3, dataset_cooling_theory['log10 L_th (erg/s)'])
sx    = np.linspace(-2, 3.51, 30)
sy    = sp(sx)

fig = plt.figure(figsize=(12.4,9.0))
# x:y = 500 : 620 = 10:12.4
plt.subplots_adjust(left=0.10, right=0.96, top=0.96, bottom=0.12)
ax  = fig.add_subplot(1,1,1)

ax.plot(np.array(sx),np.array(sy),color="#808080",linewidth=5)

ax.scatter(np.array(nth_age),np.array(nth_L),c="#FFFFFF",marker="d",linewidth=1,s=130,edgecolor="#808080",label='RPP Non-thermal')
ax.scatter(np.array(mag_age),np.array(mag_L),c="#FF0000",marker="o",linewidth=2,s=220,edgecolor="k",label='Magnetar')
ax.scatter(np.array(xins_age),np.array(xins_L),c="#E8A317",marker="p",linewidth=2,s=260,edgecolor="k",label='XINS')
ax.scatter(np.array(bbp_age),np.array(hbp_L),c="#8D38C9",marker="D",linewidth=2,s=190,edgecolor="k",label='HBP')
ax.scatter(np.array(cco_age),np.array(cco_L),c="#0000FF",marker="s",linewidth=2,s=190,edgecolor="k",label='CCO')
ax.scatter(np.array(rpp_age),np.array(rpp_L),c="#ADD8E6",marker="o",linewidth=2,s=190,edgecolor="k",label='RPP Thermal')

ax.set_xlabel('Age (Log t/kyr)',fontsize=24, labelpad=10)
ax.set_ylabel('Luminosity (Log $L$/(erg s$^{-1}$))',fontsize=24,labelpad=14)
ax.legend(loc='lower left',scatterpoints=1,fontsize=20)
ax.set_xlim(-1.2,5.0)
ax.set_ylim(27,38)

plt.savefig('cooling_thermal_nonthermal.pdf',bbox_inches='tight')
files.download('cooling_thermal_nonthermal.pdf')